In [1]:
%run /home/jovyan/work/model/src/model_evaluation.ipynb import ModelEvaluate
%run /home/jovyan/work/model/src/calculate_metrics.ipynb import ConfusionMatrix, AccuracyScore

In [2]:
import logging
import numpy as np
from typing import Dict, Tuple
from tensorflow.keras.models import Model

In [ ]:
def evaluate(model: Model, 
             test_data: Dict[str, Dict[str, np.ndarray]],
             metrics: Dict[str, str],
             model_type: str
             ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Handles model evaluation process
    
    Args:
        model: Trained model
        test_data: Dictionary containing test data
        metrics: Dictionary with names of loss and metrics algorithms used in model
        model_type: Type of model (regressor/classifier)
    Returns:
        Tuple[np.ndarray, np.ndarray]:
            - 'home_score' Confusion Matrix 
            - 'away_score' Confusion Matrix 
    """
    try:
        logging.info("\nStarted model evaluation")
        evaluate = ModelEvaluate(model=model, test_dataset=test_data)
        home_pred, away_pred = evaluate.model_predict()
        evaluate.calculate_metrics(metrics)

        if model_type == "regressor":
            home_pred = evaluate.round_results(home_pred)
            away_pred = evaluate.round_results(away_pred)
            accuracy = AccuracyScore()
            accuracy.calculate_scores(test_data["targets"]["home_score"], home_pred, "home_score")
            accuracy.calculate_scores(test_data["targets"]["away_score"], away_pred, "away_score")
        elif model_type == "classifier":
            home_pred = evaluate.select_one_category(home_pred)
            away_pred = evaluate.select_one_category(away_pred)
        else:
            raise ValueError("Wrong model_type variable, acceptable only: 'regressor' or 'classifier'")

        conf_matrix = ConfusionMatrix()
        home_conf_matrix = conf_matrix.calculate_scores(test_data["targets"]["home_score"], home_pred)
        away_conf_matrix = conf_matrix.calculate_scores(test_data["targets"]["away_score"], away_pred)
        
        logging.info("Successfully finished model evaluation \n")
        return home_conf_matrix, away_conf_matrix
    except Exception as e:
        logging.error(f"Error in evaluation process: {e}")
        raise e